# Degasperis-Procesi equation

In [ ]:
include("setup.jl")

# Periodic boundary conditions

In [ ]:
function relaxation_functional(γ, unew, uold, param)
    @unpack D, D2, inv4mD2, tmp1, tmp2, tmp3 = param
    
    @. tmp3 = (1-γ)*uold + γ*unew
    ldiv!(tmp1, inv4mD2, tmp3)
    mul!(tmp2, D2, tmp3)
    @. tmp1 = tmp1 * (tmp3 - tmp2)
    energy = integrate(tmp1, D)
end
function relaxation_functional(u, param)
    @unpack D, D2, inv4mD2, tmp1, tmp2 = param
    
    ldiv!(tmp1, inv4mD2, u)
    mul!(tmp2, D2, u)
    @. tmp1 = tmp1 * (u - tmp2)
    energy = integrate(tmp1, D)
end


function save_func_dp_periodic(u, t, integrator)
    @unpack D, D2, inv4mD2, tmp1, tmp2, usol = integrator.p
    print(".")
    
    mul!(tmp2, D2, u)
    @. tmp2 = u - tmp2
    linear = integrate(tmp2, D)
    
    ldiv!(tmp1, inv4mD2, u)
    @. tmp1 = tmp1 * tmp2
    quadratic = integrate(tmp1, D)
    
    @. tmp1 = u^3
    cubic = integrate(tmp1, D)
    
    x = grid(D)
    tmp1 .= ( usol.(t, x, x[1], -x[1]) .- u ).^2
    error_u = integrate(tmp1, D) |> sqrt
    
    SVector(linear, quadratic, cubic, error_u)
end


function dp_periodic!(du, u, param, t)
    @unpack D, D2, invImD2, tmp1, tmp2 = param
    four = 4*one(eltype(D))
    one_third = one(eltype(D)) / 3
    
    # conservative semidiscretization
    mul!(du, D, u)
    @. tmp2 = u^2
    mul!(tmp1, D, tmp2)
    @. tmp2 = -one_third * (tmp1 + u * du)
    mul!(tmp1, D2, tmp2)
    @. tmp1 = 4 * tmp2 - tmp1
    ldiv!(du, invImD2, tmp1)
    
    nothing
end

function solve_ode_dp_periodic(usol, D, D2, tspan, alg, tol, dt, adaptive)
    invImD2 = isa(D2, AbstractMatrix) ? factorize(I - D2) : I - D2
    inv4mD2 = isa(D2, AbstractMatrix) ? factorize(4*I - D2) : 4*I - D2
    
    x = grid(D)
    u0 = usol.(tspan[1], x, x[1], -x[1])
    tmp1 = similar(u0); tmp2 = similar(tmp1); tmp3 = similar(tmp1)
    param = (D=D, D2=D2, invImD2=invImD2, inv4mD2=inv4mD2, tmp1=tmp1, tmp2=tmp2, tmp3=tmp3, usol=usol)

    ode = ODEProblem(dp_periodic!, u0, tspan, param)
    
    saveat = range(tspan..., length=100)
    saved_values_baseline = SavedValues(eltype(D), SVector{4,eltype(D)})
    saving_baseline = SavingCallback(save_func_dp_periodic, saved_values_baseline, saveat=saveat)
    saved_values_relaxation = SavedValues(eltype(D), SVector{4,eltype(D)})
    saving_relaxation = SavingCallback(save_func_dp_periodic, saved_values_relaxation, saveat=saveat)
    relaxation = DiscreteCallback((u,t,integrator) -> true, relaxation!, save_positions=(false,false))
    cb_baseline = CallbackSet(saving_baseline)
    cb_relaxation = CallbackSet(relaxation, saving_relaxation)
    
    sol_relaxation = solve(ode, alg, abstol=tol, reltol=tol, dt=dt, adaptive=adaptive, save_everystep=false,
        callback=cb_relaxation, tstops=saveat)
    flush(stdout)
    sol_baseline = solve(ode, alg, abstol=tol, reltol=tol, dt=dt, adaptive=adaptive, save_everystep=false,
        callback=cb_baseline, tstops=saveat)
    flush(stdout)

    unum_baseline   = sol_baseline[end]
    unum_relaxation = sol_relaxation[end]
    uana = usol.(tspan[end], x, x[1], -x[1])
    @printf("Error in u (baseline):   %.3e\n", integrate(u->u^2, unum_baseline - uana, D) |> sqrt)
    @printf("Error in u (relaxation): %.3e\n", integrate(u->u^2, unum_relaxation - uana, D) |> sqrt)
    @printf("Difference of baseline and relaxation in u: %.3e\n", 
        integrate(u->u^2, unum_baseline - unum_relaxation, D) |> sqrt)

    sleep(0.1)
    fig_u, ax = plt.subplots(1, 1)
    plt.plot(x, u0, label=L"u^0")
    plt.plot(x, uana, label=L"$u^\mathrm{ana}$")
    plt.plot(x, unum_baseline, label=L"$u^\mathrm{num}$ (non-conservative)")
    plt.plot(x, unum_relaxation, label=L"$u^\mathrm{num}$ (conservative)")
    plt.xlabel(L"x"); plt.ylabel(L"u")
    plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5));

    t_baseline = saved_values_baseline.t
    t_relaxation = saved_values_relaxation.t
    linear_baseline      = map(x->x[1], saved_values_baseline.saveval)
    linear_relaxation    = map(x->x[1], saved_values_relaxation.saveval)
    quadratic_baseline   = map(x->x[2], saved_values_baseline.saveval)
    quadratic_relaxation = map(x->x[2], saved_values_relaxation.saveval)
    cubic_baseline       = map(x->x[3], saved_values_baseline.saveval)
    cubic_relaxation     = map(x->x[3], saved_values_relaxation.saveval)

    fig_invariants, ax = plt.subplots(1, 1)
    ax.set_yscale("symlog", linthreshy=1.0e-14)
    plt.plot(t_baseline,   linear_baseline   .- linear_baseline[1], 
        label=L"$\int (u - \partial_x^2 u)$ (non-conservative)", color="#E69F00", linestyle="-")
    plt.plot(t_relaxation, linear_relaxation .- linear_relaxation[1], 
        label=L"$\int (u - \partial_x^2 u)$ (conservative)", color="#56B4E9", linestyle="-")
    plt.plot(t_baseline,   quadratic_baseline   .- quadratic_baseline[1], 
        label=L"$\int (u - \partial_x^2 u) (4 - \partial_x^2)^{-1} u$ (non-conservative)", color="#E69F00", linestyle="--")
    plt.plot(t_relaxation, quadratic_relaxation .- quadratic_relaxation[1], 
        label=L"$\int (u - \partial_x^2 u) (4 - \partial_x^2)^{-1} u$ (conservative)", color="#56B4E9", linestyle="--")
    plt.plot(t_baseline,   cubic_baseline   .- cubic_baseline[1], 
        label=L"$\int u^3$ (non-conservative)", color="#E69F00", linestyle=":")
    plt.plot(t_relaxation, cubic_relaxation .- cubic_relaxation[1], 
        label=L"$\int u^3$ (conservative)", color="#56B4E9", linestyle=":")
    plt.xlabel(L"t"); plt.ylabel("Change of Invariants")
    plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5))
    
    error_u_baseline   = map(x->x[end], saved_values_baseline.saveval)
    error_u_relaxation = map(x->x[end], saved_values_relaxation.saveval)

    fig_error, ax = plt.subplots(1, 1)
    ax.set_xscale("log")
    ax.set_yscale("log")
    plt.plot(t_baseline,   error_u_baseline, 
        label=L"Error of $u$ (non-conservative)", color="#E69F00", linestyle="-")
    plt.plot(t_relaxation, error_u_relaxation, 
        label=L"Error of $u$ (conservative)", color="#56B4E9", linestyle="-")
    plt.xlabel(L"t"); plt.ylabel("Error")
    plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5))
    
    (; sol_relaxation, sol_baseline, saved_values_baseline, saved_values_relaxation, 
        fig_u, fig_invariants, fig_error)
end


In [ ]:
# traveling wave solution obtained numerically
c, data = open("dp_traveling_wave_init_c12_ah01_l40_N65536.txt", "r") do io
    line = readline(io)
    line = readline(io)
    line = readline(io)
    c = parse(Float64, line[8:end])
    data = readdlm(io, comments=true)
    c, data
end
x = range(data[1,1], data[end,1], length=size(data,1))
xmin = x[1]; xmax = x[end]
u0 = data[:, 2]
u0itp = CubicSplineInterpolation((x,), u0, extrapolation_bc=Periodic())
get_c() = c
function usol(t, x, xmin, xmax)
    c = get_c()
    x_t = mod(x - c*t - xmin, xmax - xmin) + xmin
    u0itp(x_t)
end


In [ ]:
println("c = ", get_c())
println("xmin = ", xmin)
println("xmax = ", xmax)
@show usol(0., xmin, xmin, xmax)
@show usol(0., xmax, xmin, xmax)
@show N = 2^8
@show dt = 0.5 * (xmax - xmin) / (N * get_c())
flush(stdout)

tspan = (0.0, (xmax-xmin)/(3*get_c()) + 40*(xmax-xmin)/get_c())

tol = 1.0e-5
adaptive = true
D = fourier_derivative_operator(xmin, xmax, N)
D2 = D^2

results = solve_ode_dp_periodic(usol, D, D2, tspan, Tsit5(), tol, dt, adaptive);

In [ ]:
ax = results.fig_error.axes[1]
t = [1.0e2, 2.0e3]
ax.plot(t, 2.0e-6 .* t.^2, ":", color="gray")
ax.annotate(L"\mathcal{O}(t^{2})", (1.0e2, 1.5e-1), color="gray")
ax.plot(t, 3.0e-6 .* t.^1, ":", color="gray")
ax.annotate(L"\mathcal{O}(t^{1})", (1.0e2, 1.5e-4), color="gray")

results.fig_error.savefig(joinpath(dirname(@__DIR__), "figures", "dp_error.pdf"), 
    bbox_inches="tight")